In [1]:
import rocksdb
db = rocksdb.DB("out/test.db", rocksdb.Options(create_if_missing=True))
db.put(b'a', b'data')
print(db.get(b'a'))

b'data'


In [8]:
import os
os.path.basename('out/test.db')

'test.db'

In [20]:
# The following example python merge operator implements a counter

class AssocCounter(rocksdb.interfaces.AssociativeMergeOperator):
    def merge(self, key, existing_value, value):
        if existing_value:
            s = int(existing_value) + int(value)
            return (True, str(s).encode('ascii'))
        return (True, value)

    def name(self):
        return b'AssocCounter'


opts = rocksdb.Options()
opts.create_if_missing = True
opts.merge_operator = AssocCounter()
db = rocksdb.DB('out/test_merge.db', opts)

db.merge(b"a", b"1")
db.merge(b"a", b"1")

# prints b'2'
print(db.get(b"a"))

b'6'


In [24]:
import gc
del db
gc.collect()

72

In [7]:
# We provide a set of default operators rocksdb.merge_operators.UintAddOperator and rocksdb.merge_operators.StringAppendOperator:

from rocksdb.merge_operators import UintAddOperator, StringAppendOperator
import struct

opts = rocksdb.Options()
opts.create_if_missing = True
# you should also play with StringAppendOperator
opts.merge_operator = UintAddOperator()
db = rocksdb.DB('out/test_uint.db', opts)
db.put(b'a', struct.pack('Q', 5566))
for x in range(1000):
    db.merge(b"a", struct.pack('Q', x))
# self.assertEqual(5566 + sum(range(1000)), struct.unpack('Q', self.db.get(b'a'))[0])
ret=struct.unpack('Q', db.get(b'a'))[0]
print(ret)
print(5566 + sum(range(1000)))

505066
505066


In [6]:
del db
gc.collect()

505

## PrefixExtractor
According to Prefix API a prefix_extractor can reduce IO for scans within a prefix range. A python prefix extractor must implement the rocksdb.interfaces.SliceTransform interface.

The following example presents a prefix extractor of a static size. So always the first 5 bytes are used as the prefix


In [1]:
import rocksdb
import gc
from itertools import takewhile
class StaticPrefix(rocksdb.interfaces.SliceTransform):
    def name(self):
        return b'static'

    def transform(self, src):
        return (0, 5)

    def in_domain(self, src):
        return len(src) >= 5

    def in_range(self, dst):
        return len(dst) == 5

opts = rocksdb.Options()
opts.create_if_missing=True
opts.prefix_extractor = StaticPrefix()

db = rocksdb.DB('out/test_prefix.db', opts)

db.put(b'00001.x', b'x')
db.put(b'00001.y', b'y')
db.put(b'00001.z', b'z')

db.put(b'00002.x', b'x')
db.put(b'00002.y', b'y')
db.put(b'00002.z', b'z')

db.put(b'00003.x', b'x')
db.put(b'00003.y', b'y')
db.put(b'00003.z', b'z')

prefix = b'00002'

it = db.iteritems()
it.seek(prefix)

# prints {b'00002.z': b'z', b'00002.y': b'y', b'00002.x': b'x'}
print(dict(takewhile(lambda item: item[0].startswith(prefix), it)))

{b'00002.x': b'x', b'00002.y': b'y', b'00002.z': b'z'}


In [2]:
db.put(b'00003.z', b'z')

In [2]:
# close
del db
gc.collect()

0

In [32]:
for e in (takewhile(lambda x: x<5, [1,4,6,4,1])):
    print(e)

1
4


⊕ [7.3. struct — Interpret strings as packed binary data — Python 2.7.16 documentation](https://docs.python.org/2/library/struct.html)


In [7]:
import struct
print(len(struct.pack('Q', 5)))
print(len(struct.pack('Q', 500000)))

8
8


In [1]:
from sagas.storage.data_space import DataSpace, sys_db
def int_to_bytes(ob):
    return str(ob).encode('ascii')

sys_db.put(b'obj', b'property', b'val')
sys_db.get(b'obj', b'property')

b'val'

In [2]:
sys_db.all_column_values(b'property')

[((<ColumnFamilyHandle name: b'property', id: 1, state: valid>, b'obj'),
  b'val')]

In [2]:
import rocksdb
import os
rocksdb.list_column_families(
        os.path.join('./db', 'sys.db'),
        rocksdb.Options(),
    )

[b'default', b'property', b'value']

In [3]:
os.path.exists(os.path.join('./db', 'sys.db'))

True

In [6]:
new_cf_column_family = sys_db.db.get_column_family(b'value')
sys_db.db.put( (new_cf_column_family, b'key'), b'value' )
print( sys_db.db.get( (new_cf_column_family, b'key') ) ) # b'value'


b'value'


In [8]:
import rocksdb

default_column_family = sys_db.db.get_column_family(b'default')

batch = rocksdb.WriteBatch()
batch.put( (default_column_family, b'key2'), b'value2' )
batch.put( (new_cf_column_family, b'key3'), b'value3' )
batch.delete( (default_column_family, b'key') ) 

sys_db.db.write(batch)

In [1]:
from simple_pb2 import MyObj, Foo
obj=MyObj(name='hello')
print(obj)

name: "hello"



In [4]:
from sagas.storage.data_space import DataSpace, sys_db
new_cf_column_family = sys_db.db.get_column_family(b'value')

sys_db.db.put( (new_cf_column_family, b'key'), obj.SerializeToString() )
val= sys_db.db.get( (new_cf_column_family, b'key') )
obj=MyObj()
obj.ParseFromString(val)
print(obj)

name: "hello"

